# Assignment 4

## Convert Line shapes to EPSG:4326

In [2]:
import json

import shapefile
from numpy import mean, median
from pyproj import Proj, transform
from tqdm.notebook import tqdm

In [3]:
# already run if you have shapes_lat_long.json
lines = shapefile.Reader('../data/raw/shapefiles/ACTU_LINES')
lambert = Proj('EPSG:31370')
latlong = Proj('EPSG:4326')
shapes = {f'{line.record["LIGNE"]}-{line.record["VARIANTE"]}': [transform(lambert, latlong, x, y) for x, y in
                                                                line.shape.points] for line in
          tqdm(lines.shapeRecords())}
with open('../data/raw/shapes_lat_long.json', 'w', encoding='utf8') as json_file:
    json.dump(shapes, json_file)

  0%|          | 0/174 [00:00<?, ?it/s]

/tmp/ipykernel_9052/3003335364.py:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shapes = {f'{line.record["LIGNE"]}-{line.record["VARIANTE"]}': [transform(lambert, latlong, x, y) for x, y in


KeyboardInterrupt: 

## Create plots for each track and line

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from tqdm.notebook import tqdm

In [8]:
tracks = pd.read_csv('../data/raw/GPSTracksAssignment4/GPStracks.csv')
with open('../data/raw/shapes_lat_long.json', 'r', encoding='utf8') as json_file:
    shapes = json.load(json_file)
for track_id in tqdm(tracks['TrackId'].unique()):
    track = tracks[tracks['TrackId'] == track_id].sort_values(by='time')
    if not os.path.exists(f'../data/line_plots/Track{track_id}'):
        os.mkdir(f'../data/line_plots/Track{track_id}')
    for index, (line_id, line) in tqdm(enumerate(shapes.items())):
        plt.scatter(x=track['lon'], y=track['lat'], label=f'Track {track_id}')
        plt.scatter(x=[lon for _, lon in line], y=[lat for lat, _ in line], label=line_id)
        plt.title(line_id)
        plt.savefig(f'../data/line_plots/Track{track_id}/{line_id}.png')
        plt.close()

  0%|          | 0/9 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

### Track ⟶ Line:

 - Track 1: Other
 - Track 3: Bus 50 or Tram 82 or Bus 49
 - Track 4: Bus 50
 - Track 5: Tram 82 or Tram 97
 - Track 6: Tram 8
 - Track 7: Tram 7
 - Track 8: Other
 - Track 10: Tram 8 or Tram 93
 - Track 11: Tram 25

## Create CSV of labeled points

In [9]:
import json
from scripts.helpers import write_csv

In [ ]:
# creating the dataset.

In [11]:
with open('../data/raw/shapes_lat_long.json', 'r', encoding='utf8') as json_file:
    shapes = json.load(json_file)
with write_csv('../data/processed/assignment4/labeled_line_points.csv') as csv_out:
    csv_out.writerow(['Lat', 'Long', 'LineId'])
    for line_id, points in shapes.items():
        if line_id[:-2] in ['032t']:  # Remove lines that are not active anymore
            continue
        for lat, long in points:
            csv_out.writerow([lat, long, line_id[:-2]])

## Create Classification Model

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from scripts.helpers import read_csv_list
import pandas as pd
from typing import Tuple, List

In [3]:
def get_datasets() -> Tuple[List[List[float]], List[str], pd.DataFrame]:
    csv_lines = read_csv_list('../data/processed/assignment4/labeled_line_points.csv')
    data_set = [[float(point[0]), float(point[1])] for point in csv_lines[1:]]
    data_labels = [point[-1] for point in csv_lines[1:]]
    tracks = pd.read_csv('../data/raw/GPStracks.csv')
    return data_set, data_labels, tracks

### Create "Ensemble" with KNN using average probability

In [4]:
data_set, data_labels, tracks = get_datasets()
print('Training model')
model = KNeighborsClassifier(n_neighbors=10)
model.fit(data_set, data_labels)
print('Model trained')
for track_id in tracks['TrackId'].unique():
    track_points = tracks[tracks['TrackId'] == track_id][['lat', 'lon']]
    predictions = model.predict_proba(track_points)
    final_predictions = [(sum(predictions[i][x]
                              for i in range(len(predictions))) * 100 / len(predictions), model.classes_[x])
                         for x in range(len(predictions[0]))]
    sorted_predictions = sorted(final_predictions, key=lambda x: x[0], reverse=True)

    print(track_id, '->', sorted_predictions[:4])

Training model
Model trained
1 -> [(41.68539325842696, '046b'), (11.96629213483146, '051t'), (11.23595505617978, '088b'), (7.808988764044942, '001m')]
3 -> [(33.13043478260868, '082t'), (23.652173913043487, '049b'), (23.391304347826086, '050b'), (10.608695652173912, '048b')]
4 -> [(36.639004149377605, '050b'), (14.190871369294602, '082t'), (14.06639004149377, '097t'), (13.775933609958502, '074b')]
5 -> [(36.788990825688074, '097t'), (36.19266055045872, '082t'), (15.229357798165125, '050b'), (8.532110091743121, '052b')]
6 -> [(42.382671480144396, '008t'), (27.400722021660652, '093t'), (7.725631768953068, '007t'), (5.9205776173285205, '071b')]
7 -> [(47.53846153846154, '007t'), (44.564102564102555, '025t'), (4.153846153846154, '008t'), (1.2307692307692308, '034b')]
8 -> [(15.318471337579615, '060b'), (14.872611464968156, '007t'), (12.229299363057322, '008t'), (11.910828025477711, '038b')]
10 -> [(42.58064516129032, '008t'), (42.58064516129032, '093t'), (6.29032258064516, '038b'), (6.2903

C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature nam

### Create "Ensemble" with KNN using "presence"

In [5]:
data_set, data_labels, tracks = get_datasets()
print('Training model')
model = KNeighborsClassifier(n_neighbors=15)
model.fit(data_set, data_labels)

track_predictions = {}

print('Model trained')
for track_id in tracks['TrackId'].unique():
    track_points = tracks[tracks['TrackId'] == track_id][['lat', 'lon']]
    predictions = model.predict_proba(track_points)
    converted_predictions = [
        [1 if probability > 0 else 0 for probability in prediction]
        for prediction in predictions
    ]
    final_predictions = [(sum(converted_predictions[i][x]
                              for i in range(len(predictions))) * 100 / len(predictions), model.classes_[x])
                         for x in range(len(predictions[0]))]
    filtered_predictions = [
        (percentage, line) for percentage, line in final_predictions if percentage > 75
    ]

    filtered_predictions = sorted(filtered_predictions, key=lambda x: x[0], reverse=True)

    if len(filtered_predictions):
        filtered_predictions = [
            (prob, line) for prob, line in filtered_predictions if filtered_predictions[0][0] - prob < 5
        ]
    track_predictions[track_id] = filtered_predictions[:4]
    print(track_id, '->', filtered_predictions[:4])


Training model
Model trained
1 -> []
3 -> [(97.3913043478261, '049b'), (96.52173913043478, '050b'), (95.65217391304348, '082t')]
4 -> [(100.0, '050b')]
5 -> [(98.62385321100918, '097t'), (98.1651376146789, '082t')]
6 -> [(97.83393501805054, '008t')]
7 -> [(100.0, '007t')]
8 -> []
10 -> [(93.54838709677419, '008t'), (93.54838709677419, '093t')]
11 -> [(100.0, '025t')]


C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
C:\Users\Victor\.virtualenvs\STIB-Hach-My-Ride-39xoxc1Q\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature nam

In [106]:
import datetime
from scripts.helpers import distance
import pandas as pd
from numpy import mean, median

In [107]:
def calculate_average_speed_of_track(track: pd.DataFrame) -> float:
    total_time = 0.0
    total_distance = 0.0
    previous_time = datetime.datetime.fromisoformat(track.iloc[0]['time'][:-1])
    previous_position = (track.iloc[0]['lat'], track.iloc[0]['lon'])
    for _, _, lat, long, time in track.sort_values(by='time').itertuples():
        timestamp = datetime.datetime.fromisoformat(time[:-1])
        seconds = (timestamp - previous_time).total_seconds()
        if seconds > 0:
            dist = abs(distance(*previous_position, lat, long)) * 1000
            total_distance += dist
            total_time += seconds
        previous_time = timestamp
        previous_position = (lat, long)
    return total_distance / total_time

In [108]:
tracks = pd.read_csv('../data/raw/GPStracks.csv')
tracks_speed = {}
for track_id in tracks['TrackId'].unique():
    track = tracks[tracks['TrackId'] == track_id]
    speed = calculate_average_speed_of_track(track)
    tracks_speed[track_id] = speed * 3.6
    print(track_id, '->', speed * 3.6, 'km/h')

1 -> 7.116623587308064 km/h
3 -> 23.12644839194265 km/h
4 -> 13.695407924105305 km/h
5 -> 14.871617291497595 km/h
6 -> 14.471340208418587 km/h
7 -> 16.346108370386162 km/h
8 -> 22.46110292777329 km/h
10 -> 51.632187693164 km/h
11 -> 25.019532962404657 km/h


In [109]:
stops = pd.read_csv('../data/processed/assignment1/line_stops.csv')


def get_closest_stop_in_direction(position, line_direction_stops):
    distances = line_direction_stops.apply(
        lambda row: distance(row['stop_lat'], row['stop_lon'], position['lat'], position['lon']), axis=1)
    min_distance = distances.min()
    return line_direction_stops[distances == min_distance].iloc[0]


def get_closest_stops_in_line(position, line_stops):
    return (get_closest_stop_in_direction(position, line_stops[line_stops['direction'] == 1]),
            get_closest_stop_in_direction(position, line_stops[line_stops['direction'] == 2]))


In [110]:
def select_line_by_speed(track, track_speed, possible_lines, speeds):
    selected_line = None
    min_dif = 99999
    for _, line_id in possible_lines:
        first_stop, last_stop = get_first_and_last_stops(line_id, track)
        median_speed = get_speed_of_line_between_stops(first_stop, last_stop, line_id, speeds)
        if abs(median_speed - track_speed) < min_dif:
            min_dif = abs(median_speed - track_speed)
            selected_line = line_id
    return selected_line


def get_speed_of_line_between_stops(first_stop, last_stop, line_id, speeds):
    line_speeds = speeds[speeds['LineId'] == int(line_id[:-1])]
    total_speed = []
    current_stop = int(first_stop['stop_id'].strip('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM'))
    while current_stop != int(last_stop['stop_id'].strip('qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM')):
        row = line_speeds[line_speeds['FromStop'] == current_stop].iloc[0]
        total_speed.append(row['speed9'])
        current_stop = row['ToStop']
    median_speed = median(total_speed)
    return median_speed


def get_first_and_last_stops(line_id, track):
    line_stops = stops[stops['lineId'] == line_id]
    first_stops = get_closest_stops_in_line(track.iloc[0], line_stops)
    last_stops = get_closest_stops_in_line(track.iloc[-1], line_stops)
    if first_stops[0]['order'] < last_stops[0]['order']:
        first_stop = first_stops[0]
        last_stop = last_stops[0]
    else:
        first_stop = first_stops[1]
        last_stop = last_stops[1]
    return first_stop, last_stop

In [112]:
def get_type_of_line(line):
    return {'m': 'Metro', 't': 'Tram', 'b': 'Bus'}[line[-1]]


speeds = pd.read_csv('../data/processed/assignment1/vehicleSpeed.csv')

for track_id in tracks['TrackId'].unique():
    track = tracks[tracks['TrackId'] == track_id]
    possible_lines = track_predictions[track_id]
    if len(possible_lines) == 0:
        print(f'{track_id} ==> Other')
    elif len(possible_lines) == 1:
        print(f'{track_id} ==> {get_type_of_line(possible_lines[0][1])} ({possible_lines[0][1]})')
    elif len(possible_lines) > 1 and len({l[1][-1] for l in possible_lines}) == 1:
        selected_line = select_line_by_speed(track, tracks_speed[track_id], possible_lines, speeds)
        print(
            f'{track_id} ==> {get_type_of_line(possible_lines[0][1])} ({" || ".join([l[1] for l in possible_lines])}) [ ==> {get_type_of_line(selected_line)} ({selected_line})]')
    else:
        possible_types = {get_type_of_line(l[1]) for l in possible_lines}
        selected_line = select_line_by_speed(track, tracks_speed[track_id], possible_lines, speeds)
        print(f'{track_id} ==> {" or ".join(possible_types)} ({" || ".join([l[1] for l in possible_lines])}) ==> {get_type_of_line(selected_line)} ({selected_line})')

1 ==> Other
3 ==> Tram or Bus (049b || 050b || 082t) ==> Tram (082t)
4 ==> Bus (050b)
5 ==> Tram (097t || 082t) [ ==> Tram (082t)]
6 ==> Tram (008t)
7 ==> Tram (007t)
8 ==> Other
10 ==> Tram (008t || 093t) [ ==> Tram (008t)]
11 ==> Tram (025t)
